## Corpora and Vector Spaces 

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from gensim import corpora, models, similarities

C:\Users\sun\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-06-03 11:41:01,275 : INFO : 'pattern' package not found; tag filters are not available for English


##### From Strings to Vectors


In [3]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [4]:
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents]

In [5]:
texts

[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'],
 ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'management', 'system'],
 ['system', 'human', 'system', 'engineering', 'testing', 'eps'],
 ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'],
 ['generation', 'random', 'binary', 'unordered', 'trees'],
 ['intersection', 'graph', 'paths', 'trees'],
 ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'],
 ['graph', 'minors', 'survey']]

In [6]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1]
        for text in texts]

In [7]:
from pprint import pprint
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [8]:
dictionary = corpora.Dictionary(texts)
dictionary.save('./data/deerwester.dict')
print(dictionary)

2018-06-03 11:41:22,698 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-06-03 11:41:22,698 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2018-06-03 11:41:22,699 : INFO : saving Dictionary object under ./data/deerwester.dict, separately None
2018-06-03 11:41:22,704 : INFO : saved ./data/deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [9]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [10]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (1, 1)]


In [11]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('./data/deerwester.mm', corpus)

2018-06-03 11:44:00,809 : INFO : storing corpus in Matrix Market format to ./data/deerwester.mm
2018-06-03 11:44:00,822 : INFO : saving sparse matrix to ./data/deerwester.mm
2018-06-03 11:44:00,822 : INFO : PROGRESS: saving document #0
2018-06-03 11:44:00,823 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2018-06-03 11:44:00,827 : INFO : saving MmCorpus index to ./data/deerwester.mm.index


For people familiar with scikit learn, doc2bow() has similar behaviors as calling transform() on CountVectorizer

In [21]:
pprint(corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


#### Corpus Streaming – One Document at a Time


In [17]:
class MyCorpus(object):
    def __init__(self, f):
        self.f = f
    def __iter__(self):
        for line in open(self.f):
            yield dictionary.doc2bow(line.lower().split())

In [18]:
corpus_memory_friendly = MyCorpus('./data/mycorpus.txt')
print(corpus_memory_friendly)

In [19]:
for vector in corpus_memory_friendly:
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


Another way to process the text. Similarly, to construct the dictionary without loading all texts into memory:

In [22]:
from six import iteritems
dictionary = corpora.Dictionary(line.lower().split() for line in open('./data/mycorpus.txt'))
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
           if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs)
           if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)
dictionary.compactify()
print(dictionary)

2018-06-03 12:04:09,734 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-06-03 12:04:09,735 : INFO : built Dictionary(42 unique tokens: ['abc', 'applications', 'computer', 'for', 'human']...) from 9 documents (total 69 corpus positions)


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


## Topics and Transformations 

In [23]:
import os
if(os.path.exists('./data/deerwester.dict')):
    dictionary = corpora.Dictionary.load('./data/deerwester.dict')
    corpus = corpora.MmCorpus('./data/deerwester.mm')
    print("used files generated from first tutorial")
else:
    print("Please run first tutorial to generate the set")

2018-06-03 15:30:08,374 : INFO : loading Dictionary object from ./data/deerwester.dict
2018-06-03 15:30:08,375 : INFO : loaded ./data/deerwester.dict
2018-06-03 15:30:08,376 : INFO : loaded corpus index from ./data/deerwester.mm.index
2018-06-03 15:30:08,376 : INFO : initializing cython corpus reader from ./data/deerwester.mm
2018-06-03 15:30:08,377 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


used files generated from first tutorial


In [24]:
tfidf = models.TfidfModel(corpus)

2018-06-03 15:30:09,483 : INFO : collecting document frequencies
2018-06-03 15:30:09,486 : INFO : PROGRESS: processing document #0
2018-06-03 15:30:09,488 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [25]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) 

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [26]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [27]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi[corpus_tfidf]

2018-06-03 15:30:25,187 : INFO : using serial LSI version on this node
2018-06-03 15:30:25,187 : INFO : updating model with new documents
2018-06-03 15:30:25,189 : INFO : preparing a new chunk of documents
2018-06-03 15:30:25,189 : INFO : using 100 extra samples and 2 power iterations
2018-06-03 15:30:25,190 : INFO : 1st phase: constructing (12, 102) action matrix
2018-06-03 15:30:25,190 : INFO : orthonormalizing (12, 102) action matrix
2018-06-03 15:30:25,213 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-06-03 15:30:25,228 : INFO : computing the final decomposition
2018-06-03 15:30:25,229 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2018-06-03 15:30:25,232 : INFO : processed documents up to #9
2018-06-03 15:30:25,233 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2018-06-03 15:30:25,234 : INFO : topic #

In [28]:
lsi.print_topics(2)

2018-06-03 15:30:44,372 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2018-06-03 15:30:44,372 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [29]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc in corpus_lsi:
    print(doc)

[(0, 0.0660078339609046), (1, -0.5200703306361845)]
[(0, 0.19667592859142663), (1, -0.7609563167700046)]
[(0, 0.08992639972446548), (1, -0.7241860626752509)]
[(0, 0.07585847652178265), (1, -0.6320551586003429)]
[(0, 0.1015029918498025), (1, -0.5737308483002957)]
[(0, 0.7032108939378305), (1, 0.16115180214025926)]
[(0, 0.8774787673119824), (1, 0.16758906864659573)]
[(0, 0.9098624686818569), (1, 0.14086553628719178)]
[(0, 0.6165825350569278), (1, -0.05392907566389278)]


In [30]:
lsi.save('./data/model.lsi')
lsi = models.LsiModel.load('./data/model.lsi')

2018-06-03 15:32:16,067 : INFO : saving Projection object under ./data/model.lsi.projection, separately None
2018-06-03 15:32:16,072 : INFO : saved ./data/model.lsi.projection
2018-06-03 15:32:16,072 : INFO : saving LsiModel object under ./data/model.lsi, separately None
2018-06-03 15:32:16,073 : INFO : not storing attribute projection
2018-06-03 15:32:16,073 : INFO : not storing attribute dispatcher
2018-06-03 15:32:16,078 : INFO : saved ./data/model.lsi
2018-06-03 15:32:16,079 : INFO : loading LsiModel object from ./data/model.lsi
2018-06-03 15:32:16,079 : INFO : loading id2word recursively from ./data/model.lsi.id2word.* with mmap=None
2018-06-03 15:32:16,080 : INFO : setting ignored attribute projection to None
2018-06-03 15:32:16,081 : INFO : setting ignored attribute dispatcher to None
2018-06-03 15:32:16,081 : INFO : loaded ./data/model.lsi
2018-06-03 15:32:16,082 : INFO : loading LsiModel object from ./data/model.lsi.projection
2018-06-03 15:32:16,083 : INFO : loaded ./data/mod

In [31]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

2018-06-03 16:10:22,124 : INFO : using symmetric alpha at 0.01
2018-06-03 16:10:22,125 : INFO : using symmetric eta at 0.01
2018-06-03 16:10:22,126 : INFO : using serial LDA version on this node
2018-06-03 16:10:22,127 : INFO : running online (single-pass) LDA training, 100 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2018-06-03 16:10:22,127 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-06-03 16:10:22,136 : INFO : -124.785 per-word bound, 36644080589458173557036207491403218944.0 perplexity estimate based on a held-out corpus of 9 documents with 29 words
2018-06-03 16:10:22,136 : INFO : PROGRESS: pass 0, at document #9/9
2018-06-03 16:10:22,140 : INFO : topic #8 (0.010): 0.083*"user" + 0.083*"system" + 0.083*"graph" + 0.083*"trees" + 0.083*"eps" + 0.0

In [36]:
corpus_lda = lda[corpus]

In [37]:
for doc in corpus_lda:
    print(doc)

[(60, 0.7525)]
[(31, 0.85857147)]
[(35, 0.80200005)]
[(73, 0.80200005)]
[(17, 0.7525)]
[(12, 0.505)]
[(67, 0.67)]
[(47, 0.7525)]
[(81, 0.7525)]


In [39]:
hdp = models.HdpModel(corpus, id2word=dictionary)

2018-06-03 16:43:36,389 : INFO : (0, '0.253*time + 0.169*minors + 0.115*user + 0.115*human + 0.100*survey + 0.099*system + 0.042*response + 0.032*trees + 0.030*eps + 0.022*graph')
2018-06-03 16:43:36,390 : INFO : (1, '0.329*survey + 0.178*time + 0.100*user + 0.085*eps + 0.058*human + 0.055*trees + 0.043*response + 0.042*interface + 0.038*computer + 0.037*graph')
2018-06-03 16:43:36,390 : INFO : (2, '0.168*response + 0.168*minors + 0.143*trees + 0.143*survey + 0.141*human + 0.057*time + 0.043*system + 0.041*graph + 0.034*computer + 0.029*interface')
2018-06-03 16:43:36,390 : INFO : (3, '0.328*trees + 0.200*survey + 0.127*eps + 0.112*human + 0.071*time + 0.046*graph + 0.033*minors + 0.029*computer + 0.025*system + 0.020*interface')
2018-06-03 16:43:36,391 : INFO : (4, '0.254*survey + 0.172*human + 0.170*interface + 0.126*user + 0.102*trees + 0.055*time + 0.035*minors + 0.030*graph + 0.022*response + 0.019*computer')
2018-06-03 16:43:36,391 : INFO : (5, '0.329*human + 0.138*interface + 0.

In [40]:
corpus_hdp = hdp[corpus]

In [41]:
for doc in corpus_hdp:
    print(doc)

[(0, 0.06741614680567234), (1, 0.7920927532224492), (2, 0.035688734653243494), (3, 0.026529884585874488), (4, 0.019860377677366974), (5, 0.014872695960563825), (6, 0.011150500194695462)]
[(0, 0.4062036335156195), (1, 0.5135419015170811), (2, 0.020373199057003757), (3, 0.015154786977715686), (4, 0.011348147168246982)]
[(0, 0.05760860426667446), (1, 0.8301022749912499), (2, 0.02845171682189931), (3, 0.021219954643810615), (4, 0.01588777607521441), (5, 0.0118981475183838)]
[(0, 0.8491454873551538), (1, 0.03856316322806118), (2, 0.028446905373780192), (3, 0.021227456329563976), (4, 0.01588731851671879), (5, 0.0118981435379153)]
[(0, 0.8114113730097752), (1, 0.0480967088486063), (2, 0.03570462828068052), (3, 0.026515982512872547), (4, 0.019859229646241316), (5, 0.01487267060595147), (6, 0.011150500195747818)]
[(0, 0.1278178000073639), (1, 0.09535577664685667), (2, 0.5671660453117332), (3, 0.05311736296509713), (4, 0.03971885447243531), (5, 0.02974534650477296), (6, 0.02230100029150766), (7,

## Similarity Queries 

In [42]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.07910475117444982), (1, -0.5732835243079397)]


In [43]:
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

2018-06-03 17:50:31,629 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-06-03 17:50:31,630 : INFO : creating matrix with 9 documents and 2 features


In [48]:
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, 0.9999408), (1, 0.9946708), (2, 0.9999428), (3, 0.999879), (4, 0.99935204), (5, -0.08804217), (6, -0.0515742), (7, -0.023664713), (8, 0.1938726)]


In [49]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
pprint(sims) # print sorted (document number, similarity score) 2-tuples

[(2, 0.9999428),
 (0, 0.9999408),
 (3, 0.999879),
 (4, 0.99935204),
 (1, 0.9946708),
 (8, 0.1938726),
 (7, -0.023664713),
 (6, -0.0515742),
 (5, -0.08804217)]


> https://radimrehurek.com/gensim/tutorial.html